<a href="https://colab.research.google.com/github/vihaannnn/AI-Allignment-Research/blob/master/Submissions/XAI_in_Embedding_Models_for_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

XAI for embeddings of news articles based on the topic - Artificial Intelligence

Some portions of this code were generated using Perplexity AI and Gemini AI

Some functions were taken from the class Data Sourcing for Analytics - TA-0

Here we will be using the Embedding model - stella_en_400M_v5. This is a relitevely small model but has still managed to make it's way up onto the MTEB Leaderboard. It is a 435M parameter model with 8192 embedding dimensions

We will be using NewsAPI.org for data, from where we will get the top 100 articles relating to "Artificial Intelligence".

We will be creating visualization of a scaled down embedding space for the titles of these news articles, using 3 techniques - namely - t-distributed Stochastic Neighbor Embedding, Principal Component Analysis, and Uniform Manifold Approximation and Projection

Please ensure you have access to a GPU to run this code

Note - you can hver over points in the graph to note the article title

In [1]:
#Clone the project
!git clone https://huggingface.co/dunzhang/stella_en_400M_v5

Cloning into 'stella_en_400M_v5'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 84 (delta 23), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (84/84), 415.94 KiB | 2.22 MiB/s, done.
Filtering content: 100% (18/18), 3.47 GiB | 44.24 MiB/s, done.


In [2]:
#install required dependencies
!pip install sentence_transformers plotly

In [3]:
#Obtain the latest version of Pytorch
!pip install torch --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
#install dependencies
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 86.8 MB/s eta 0:00:00


In [5]:
def fetch_articles(topic):
    """
    Function to fetch articles from the news_api.py
    Input - a call from the main function
    Output - the data from the API in a list format
    """
    import os
    import requests

    # get apikey from env
    # TODO: Paste API key from https://newsapi.org/register
    api_key = '1d3841bf8a03408192466a41ba2221d1' #Enter your api key here
    # base URL
    base_url = "https://newsapi.org/v2/everything"
    # init param
    params = {
        'q': topic ,
        'apiKey': api_key,
        'language': 'en',
        'sortBy': 'relevancy'
    }
    articles = []
    try:
        # get request
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        # read json
        articles = response.json().get('articles', [])
        if articles:
            for i, article in enumerate(articles, start=0):
                if (i < 6):
                    print(f"Article {i}:")
                    print(f"Title: {article['title']}")
                    print(f"Description: {article['description']}")
                    print(f"URL: {article['url']}\n")
                    print(f"Published At: {article['publishedAt']}\n")
                else:
                    break
        else:
            print("No articles found.")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return articles

def convert_and_save_dataframe(articles, name):
    """
    Function to convert the data from the api and return it in the dataframe format
    Input - all the data in the list format, name of the csv to be saved as
    Output - the data data in dataframe format
    """
    import pandas as pd
    df = pd.DataFrame(articles)
    print("Printing out all our data from the from the API in the dataframe format")
    print(df)
    df.to_csv(name)
    return df

In [6]:
def plot_sentiment(df, x_axis, y_axis, title_graph):
    """
     The following function takes in a df, x-axis label, y-axis label and title of graph to provide a high-level analysis of embedding space through a scatter plot.

     Param: Preprocessed Dataframe, x-axis label, y-axis label and title of graph.

     Return: Interactive Scatter Plot
    """
    import pandas as pd
    import plotly.express as px
    # handle publishAt class to conversion to dateTime


    # create plot and add "hoverable" elements
    fig = px.scatter(
        df,
        x=df.columns[1],
        y=df.columns[2],
        hover_name=df.columns[0],
        # hover_data={'url': True},
        labels={
            df.columns[1]: x_axis,
            df.columns[2]: y_axis
        },
        title = title_graph
    )

    fig.show()


In [7]:
#fetch articles from newsapi
articles = fetch_articles("Artificial Intelligence")

Article 0:
Title: How a Trump Win Could Unleash Dangerous AI
Description: Donald Trump's opposition to “woke” safety standards for artificial intelligence would likely mean the dismantling of regulations that protect Americans from misinformation, discrimination, and worse.
URL: https://www.wired.com/story/donald-trump-ai-safety-regulation/

Published At: 2024-10-21T10:30:00Z

Article 1:
Title: This Toilet Seat Has a Camera for Taking Pictures of Your Poop
Description: The seat's camera uses "artificial gut intelligence" to help identify any potential digestive health issues.
URL: https://gizmodo.com/this-toilet-seat-has-a-camera-for-taking-pictures-of-your-poop-2000514322

Published At: 2024-10-21T15:19:27Z

Article 2:
Title: President Biden sets up new AI guardrails for military, intelligence agencies
Description: The White House issued its first national security memorandum outlining the use of artificial intelligence for the military and intelligence agencies. The White House also 

In [8]:
#Convert article list to dataframe
df = convert_and_save_dataframe(articles, 'articles.csv')

Printing out all our data from the from the API in the dataframe format
                                              source               author  \
0                   {'id': 'wired', 'name': 'Wired'}          Eric Geller   
1                {'id': None, 'name': 'Gizmodo.com'}           Dua Rashid   
2        {'id': None, 'name': 'Yahoo Entertainment'}      Danny Gallagher   
3                {'id': None, 'name': 'Gizmodo.com'}        Todd Feathers   
4                  {'id': None, 'name': '[Removed]'}                 None   
..                                               ...                  ...   
95  {'id': 'new-scientist', 'name': 'New Scientist'}  Chris Stokel-Walker   
96               {'id': None, 'name': 'Gizmodo.com'}           Dua Rashid   
97              {'id': None, 'name': 'Slashdot.org'}               BeauHD   
98           {'id': None, 'name': 'Theregister.com'}       Simon Sharwood   
99                 {'id': None, 'name': 'MakeUseOf'}         Philip Bates   

   

In [9]:
titles = df['title']

In [10]:
import pandas as pd

print(type(titles))
titles_list = titles.tolist()
print(type(titles_list))

<class 'pandas.core.series.Series'>
<class 'list'>


In [11]:
print(titles_list)

['How a Trump Win Could Unleash Dangerous AI', 'This Toilet Seat Has a Camera for Taking Pictures of Your Poop', 'President Biden sets up new AI guardrails for military, intelligence agencies', 'Nobel Prize Goes to ‘Godfathers of AI’ Who Now Fear Their Work Is Growing Too Powerful', '[Removed]', "Here's how Anthropic CEO Dario Amodei defines artificial general intelligence", "I Used AI to Help With Grief. Here's What I Learned", "This is OpenAI CEO Sam Altman's favorite question about AGI", 'Julian Assange Says He’s Still Getting Used to the ‘Spooky Sound of Electric Cars’', 'How to Use AI to Save Money on Groceries', "Reddit CEO says the platform is in an 'arms race' for AI training", 'Microsoft’s Copilot AI Gets a Voice, Vision, and a ‘Hype Man’ Persona', 'A Nobel Prize for Artificial Intelligence', 'Nvidia just dropped a new AI model that crushes OpenAI’s GPT-4—no big launch, just big results', 'Amazon’s New Kindle Lineup Includes the First-Ever Color Kindle', 'Marissa Mayer: I Am N

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/186 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

In [13]:
#obtain embeddings of the titles
sentences = titles_list
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [100, 100]

torch.Size([100, 100])


# Visualizing the Embeddings in 2 dimensional space

Explanations done on Oct 30, 2024 for the tpic Artificial Intelligence - this matters as the topic can change and the relevant news articles may change (based on when executed) as well - so the embeddings will change as well

## Tsne

In [14]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Assuming your_list_of_embeddings is your list of embedding vectors
embeddings_array = np.array(embeddings)

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=300)
tsne_results = tsne.fit_transform(embeddings_array)

data = {
    'Titles': titles_list,
    'T-SNE-1': tsne_results[:, 0],
    'T-SNE-2': tsne_results[:, 1]
}

df = pd.DataFrame(data)
plot_sentiment(df, "T-SNE-1", "T-SNE-2", "T-SNE of News Articles")


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


t-SNE, or t-distributed Stochastic Neighbor Embedding, is a dimensionality reduction technique primarily used for visualizing high-dimensional data in a 2D or 3D space. It’s particularly effective at preserving local structures (i.e., similar points stay close together) while capturing patterns that highlight clusters and relationships in the data.

Let us analyse the diagram we have on our screen shown -
1. in the top left of the screen we have 3 dots relitevely close to each other - these relate to apple and their updates in the field of AI. We can see that one article speaks about the Mac Mini and does not explicitly mention the word "Apple" but it still groups these together, showing the model understands the underlying realtions of these words.
2. We can see in the top - middle the documents relating to the Nobel Prize and Geoffrey Hinton are present. These seem to be grouped by the term "Godfather of AI" we can see that right around these documents are the documents relating to other developments referencing the nobel prize (the Nobel Prize in Chemistry) and also the articles that relate to physics in general as we know the nobel prize was won by geoffrey hinto in the field of physics.
3.  The middle of the graph seems to have all the news realted to big AI companies such as - OpenAI, MetaAI, Adobe etc and their news on AI. This shows that the model is able to draw parallels to the companies and place them close to each other.

## PCA

In [15]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Assuming your_list_of_embeddings is your list of embedding vectors
embeddings_array = np.array(embeddings)

# Apply PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(embeddings_array)


data = {
    'Titles': titles_list,
    'PCA-1': pca_results[:, 0],
    'PCA-2': pca_results[:, 1]
}

df = pd.DataFrame(data)
plot_sentiment(df, "PCA-1", "PCA-2", "Principal Component Analyis of News Articles")

# Print the explained variance ratio
print('Explained variance ratio:', pca.explained_variance_ratio_)

Explained variance ratio: [0.07646565 0.04890425]


## UMAP

Principal Component Analysis (PCA) is a widely used dimensionality reduction technique that transforms high-dimensional data into a lower-dimensional form while preserving as much variance (information) as possible. It’s particularly useful in machine learning, data analysis, and visualization to simplify datasets and reduce noise. Here’s an explanation of how PCA works

Let us Analyse the diagram now -
1. The embedding space seems to be far more compact. This could mean that the dimensionality reduction model is focussing a lot on outliers and not outting that much emphasis on the points that are closer together.

2. In the top left of the screen we see all the terms related to the Nobel Prize, It looks like there the dimansionality reduction model has highlighted dimensions that relate to the subject in which AI was used in the title - in this case the Nobel Prize and science (Chemistry and Physics). It is far away from the rest of the

3. In the middle-right of the graph we see those points that relate to the government and AI. We see one point highlighting Donald Trump but not mentioning the word Government in its' title close to the government/legal cluster. This indocates that the model has knowledge of proper nouns such as names and the domains they relate too as well.

In [16]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.7 MB/s eta 0:00:00


In [17]:
import numpy as np
import umap.umap_ as umap # Import UMAP from umap.umap_
import matplotlib.pyplot as plt

# Assuming your_list_of_embeddings is your list of embedding vectors
embeddings_array = np.array(embeddings)

# Apply UMAP
reducer = umap.UMAP(n_components=2) # Use umap.UMAP now
umap_results = reducer.fit_transform(embeddings_array)

data = {
    'Titles': titles_list,
    'UMAP-1': umap_results[:, 0],
    'UMAP-2': umap_results[:, 1]
}

df = pd.DataFrame(data)
plot_sentiment(df, "UMAP-1", "UMAP-2", "UMAP of News Articles")



UMAP (Uniform Manifold Approximation and Projection) is a dimensionality reduction technique used for visualizing high-dimensional data in a lower-dimensional space, typically 2D or 3D. UMAP has gained popularity in various fields, particularly in bioinformatics and data science, due to its ability to handle large datasets efficiently while preserving both local and global structures in the data.

Let us now analyse the graph -
1. We see this graph is more spread out but we can also notice that the range of units on the x-axis is much smaller than the other dimensionality reduction models we have used till now. This shows that the dimensionalty reduction model can map out tiny differences in topics as well.
2. In the top left we see the articles relating to the nobel prize all grouped together
3. In the top right we see all the data that relates to the field of AI and news especially tagging big names and even big personalities such as Elon Musk
4. In the middle left we have all the news articles from FOX. It looks like the model clustered those together not on the subject of the content in them but rather on the fact that they had the term "FOX" in it. It seems that the model gave up on the context similarity because it found one proper noun that was well mapped.